In [173]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm
import time
import KalmanFilter

In [174]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [175]:
thymio_data = []
Ts = 0.1

def motors(l_speed=500, r_speed=500):
    return {
        "motor.left.target": [l_speed],
        "motor.right.target": [r_speed],
    }

async def forward(motor_speed):
    await node.set_variables(motors(motor_speed,motor_speed))

async def rotate(theta, motor_speed): #theta is in radians
    direction_rot=(theta>=0)-(theta<0)
    await node.set_variables(motors(motor_speed*direction_rot, -motor_speed*direction_rot))
    # wait time to get theta 1.44 is the factor to correct
    time=(theta)*100/motor_speed*1.44
    await(client.sleep(time))
    # stop the robot
    await node.set_variables(motors(0, 0))

async def stop_motor():
    await node.set_variables(motors(0,0))

async def get_proximity_values():
    # Wait for the Thymio node
    node = await client.wait_for_node()
    # Wait for the proximity sensor variables
    await node.wait_for_variables({"prox.horizontal"})
    # Get the proximity values : v: Stands for "variables" and is used to access the cached variable values.
    proximity_values = node.v.prox.horizontal
    # Return the value of the front proximity sensor (index 2)
    return proximity_values[0:5]

## Parameters for local navigation
threshold_obst = 3500 
threshold_loc = 2500
local_motor_speed = 100
threshold_obst_list = [3200, 3600, 3600, 3600, 3200]

async def local_navigation():
    threshold_obst = 1000
    threshold_loc = 800
    local_motor_speed = 100
    threshold_obst_list = [3200, 3600, 3600, 3600, 3200]
    sens = await get_proximity_values()

    # Follow the obstacle by the left
    if (sens[0] + sens[1]) > (sens[4] + sens[3]):
        await bypass('right', sens, threshold_loc, local_motor_speed)

    # Follow the obstacle by the right    
    else:
        await bypass('left', sens, threshold_loc, local_motor_speed)

async def bypass(leftright, sens, threshold_loc, local_motor_speed):
    if leftright == "right":
        while sum(sens[i] > threshold_obst for i in range(0, 5)) > 0:
            print("Turn right")
            await rotate(np.pi/6/2, local_motor_speed)
            sens = await get_proximity_values()
            print(sens)

        while sens[0] > threshold_loc:
            await forward(local_motor_speed)
            sens = await get_proximity_values()
        
    elif leftright == "left":
        while sum(sens[i] > threshold_loc for i in range(0, 5)) > 0:
            print("Turn left")
            await rotate(-np.pi/6, local_motor_speed)
            sens = await get_proximity_values()

        while sens[4] > threshold_loc:
            await forward(local_motor_speed)
            sens = await get_proximity_values()

    if (leftright=="right" and sens[0] < threshold_loc):
        await forward(local_motor_speed)
        time.sleep(2)
        await rotate(-np.pi/4, 100)    

    await forward(local_motor_speed)
    await stop_motor()

def get_data():
    thymio_data.append({"sensor":node["prox.horizontal"],
                        "left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})

async def get_speed():
    await node.wait_for_variables() # wait for Thymio variables values
    rt = RepeatedTimer(Ts, get_data) # it auto-starts, no need of rt.start()

    try:
        await client.sleep(Ts)
    finally:
        rt.stop() # better in a try/finally block to make sure the program ends!
        node.send_set_variables(motors(0, 0))

def speed_estimation(left_speed, right_speed):
    real_thymio_speed = 25.5 #mm/s
    real_thymio_angular_speed = 0.38 #rad/s

    #speed estimation
    speed_measured = (right_speed + left_speed)/2
    speed = (speed_measured * real_thymio_speed) / 50
    angular_speed_measured = (right_speed - left_speed)/2
    angular_speed = (angular_speed_measured * real_thymio_angular_speed) / 50

    return speed, angular_speed

# def pos_estimation(left_speed, right_speed, dt, pos0_x, pos0_y, theta0):
    
#     speed, angular_speed = speed_estimation(left_speed, right_speed)

#     #position estimation
#     theta = theta0 + angular_speed * dt
#     pos_x = pos0_x + speed * np.cos(theta) * dt
#     pos_y = pos0_y + speed * np.sin(theta) * dt

#     return pos_x, pos_y, theta, speed, angular_speed

def pos_estimation(speed, angular_speed, dt, previous_state_estimation):
    
    A = np.identity(3)
    B = np.array([[np.cos(previous_state_estimation[2]).item() * dt, 0, 0],
                  [np.sin(previous_state_estimation[2]).item() * dt, 0, 0],
                  [0, dt, 0]])
    input_vector = np.array([[speed], [angular_speed], [0]])
    state_estimation = np.dot(A, previous_state_estimation) + np.dot(B, input_vector)

    return state_estimation

In [176]:
import numpy as np

# Définition des matrices et valeurs
Ts = 0.1
x_est = np.array([[0], [0]])
print("x_est :", x_est[-1])
y = 11.8125
H = np.array([[0, 1]])
A = np.array([[1, Ts], [0, 1]])
R = 6.153022447388423
q_nu = 0.04
qp = 0.25
P_est = [1000 * np.ones(2)]
#print("P_est :", P_est[-1])
P_est_a_priori = np.dot(A, np.dot(P_est[-1], A.T)) + np.array([[qp, 0], [0, q_nu]])

i = y - np.dot(H, x_est)
print("np.dot(H, x_est) est :")
print(np.dot(H, x_est))
# Calcul de S
S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R

# Calcul de K
K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)))
#print("x_est ", x_est)
x_est = x_est + np.dot(K,i)
print("i est :")
print(i)
# print("La matrice S est :")
# print(S)
print("La matrice K est :")
print(K)
print("np.dot(K,i) est :")
print(np.dot(K,i))
print("L'estimation de l'état est :")
print("1", x_est[0][0])
print("2", x_est[1])

x_est : [0]
np.dot(H, x_est) est :
[[0]]
i est :
[[11.8125]]
La matrice K est :
[[0.9938451 ]
 [0.99388485]]
np.dot(K,i) est :
[[11.73979518]
 [11.74026478]]
L'estimation de l'état est :
1 11.739795184892222
2 [11.74026478]


In [177]:
def kalman_filter(speed, angular_speed, position_camera_history, previous_state_estimation, previous_covariance_estimate, dt,
                  HT=None, HNT=None, RT=None, RNT=None):
    """
    Estimates the current state using input sensor data and the previous state
    
    param speed: measured speed (Thymio units)
    param angular_speed: measured angular speed (Thymio units)
    param x_prev: previous x position
    param y_prev: previous y position
    param x: current x position
    param y: current y position
    param position_camera: last position coordinates given by the camera
    param previous_state_estimation
    param previous_covariance_estimate: previous state a posteriori covariance
    
    return pos_last_trans: updated if a transition has been detected
    return x_estimate: new a posteriori state estimation
    return P_estimate: new a posteriori state covariance (incertitude)
    """

    std_speed = 12     #std_speed = np.std(speed_history)
    q_nu = std_speed / 2 # variance on speed state
    r_nu = std_speed / 2 # variance on speed measurement 

    #The standard deviation is chosen arbitrarily: √qp = 0.2
    qp = 0.04 # variance on position state
    rp = 0.25 # variance on position measurement
    #Assuming that half og the varance is caused by the measurements and half is caused by perturbations to the states
    # q_nu = speed/2 # variance on speed state
    # r_nu = speed/2 # variance on speed measurement
    q_nu = 6
    r_nu = 6
    Q = np.array([[qp, 0, 0], [0, qp, 0], [0, 0, q_nu]])     #Q = np.array([[qp, 0], [0, q_nu]])

    if len(position_camera_history) > 1:
        last_position_camera, position_camera = position_camera_history[-2], position_camera_history[-1]
    else:
        last_position_camera, position_camera = None, None

    # Initialising the remaining constants
    # units: length [mm], time [s]
    # A = np.identity(3)
    # B = np.array([[np.cos(previous_state_estimation[2]) * dt, 0], [np.sin(previous_state_estimation[2]) * dt, 0], [0, dt]])
    # input_vector = np.array([[speed], [angular_speed]])
    #B_derivative = np.array([[np.cos(previous_state_estimation[2]) * dt, 0], [np.sin(previous_state_estimation[2]) * dt, 0], [0, 0]])
    
    ## Prediciton through the a priori estimate
    #predicted_state_estimate = np.dot(A, previous_state_estimation) + np.dot(B, input_vector)
    predicted_state_estimate = pos_estimation(speed, angular_speed, dt, previous_state_estimation)
    predicted_state_estimate_derivative = np.array([[1 - angular_speed * np.sin(previous_state_estimation[2].item()) * dt, 0, 0],
                                                    [angular_speed * np.cos(previous_state_estimation[2]).item() * dt, 1, 0],
                                                    [0, 0, 1]])
    predicted_covariance_estimate = np.dot(predicted_state_estimate_derivative, np.dot(previous_covariance_estimate, predicted_state_estimate_derivative.T)) + Q
    

    ## Update         
    if last_position_camera != position_camera and position_camera is not None:
        # camera position is available
        y = np.array([position_camera])
        H = np.identity(3)
        R = np.array([[rp, 0, 0], [0, rp, 0, 0], [0, 0, rp]])
    else:
        # no camera position, use only the speed measurement
        y = np.array(pos_estimation(speed, angular_speed, dt, previous_state_estimation))
        H = np.identity(3)
        R = np.array([[r_nu, 0, 0], [0, r_nu, 0], [0, 0, r_nu]])

    # innovation / measurement residual
    i = y - np.dot(H, previous_state_estimation)
    # measurement prediction covariance
    S = np.dot(H, np.dot(previous_covariance_estimate, H.T)) + R
             
    # Kalman gain (tells how much the predictions should be corrected based on the measurements)
    K = np.dot(previous_covariance_estimate, np.dot(H.T, np.linalg.inv(S)))
    
    # Updated state and covariance estimate
    state_estimate = predicted_state_estimate + np.dot(K, i)
    #P_estimate = previous_covariance_estimate - np.dot(K, np.dot(H, previous_covariance_estimate))
    P_estimate = np.dot((np.identity(3) - np.dot(K, H)), predicted_covariance_estimate)
     
    return state_estimate, P_estimate

In [178]:
async def get_position(start_time, state_estimate, P_estimate):

    await get_speed()
    left_speed = node["motor.left.speed"]
    right_speed = node["motor.right.speed"]
    dt = time.time() - start_time 
    #speed, angular_speed, pos_x, pos_y, theta = pos_estimation(left_speed, right_speed, dt, pos_x, pos_y, theta)
    speed, angular_speed = speed_estimation(left_speed, right_speed)

    camera_position_histoty = []
    camera_position = [np.array([[0], [0], [0]])] #get_camera_position()
    camera_position_histoty.append(camera_position)
    camera_position = [np.array([[0], [0], [0]])] #get_camera_position()
    
    new_state_estimate, new_P_estimate = kalman_filter(speed, angular_speed, camera_position_histoty, state_estimate[-1], P_estimate[-1], dt)
    state_estimate.append(new_state_estimate)
    P_estimate.append(new_P_estimate)

    return state_estimate, P_estimate

In [181]:
#MAIN CODE : 

#parameters
local_obstacle = False
threshold_obst = 3500 #threshold for obstacle detection
pos_x, pos_y, theta = 0, 0, 0
speed_history = []
camera_position_histoty = []
thymio_data.clear()

async def main():

    global local_obstacle, pos_x, pos_y, theta, dt
    start_time = time.time()
    state_estimate = [np.array([[0], [0], [0]])]
    P_estimate = [1000 * np.ones(3)]

    while(1):
        sens = await get_proximity_values()
        if (sum(sens[i] > threshold_obst for i in range(0, 5)) > 0):
            local_obstacle = True

        if(local_obstacle):
            await local_navigation()
            start_time = time.time() #reset the timer          

        if not local_obstacle: 
            node.send_set_variables(motors(50, 50))
            state_estimate, P_estimate = await get_position(start_time, state_estimate, P_estimate)
            start_time = time.time()
            print(state_estimate[-1][0]/10, state_estimate[-1][1]/10, (state_estimate[-1][2]*180)/np.pi)

await main()

#speed_history.append(speed)
#std_speed = np.std(speed_history)
#print("The speed variance in mm^2/s^2 is {}".format(std_speed))

[0.] [0.] [0.]
[0.] [0.] [0.]
[0.] [0.] [0.]
[0.26165055] [0.0241332] [1.8654546]
[0.50943296] [0.05039203] [2.58286899]
[0.88260041] [0.08950154] [2.05670127]
[1.10077235] [0.10791386] [2.17096973]
[1.34757513] [0.12724072] [1.71378105]
[1.6324204] [0.14530695] [2.36076136]
[1.87287433] [0.16210755] [3.14863833]
[2.10759709] [0.18082178] [3.3343266]
[2.45918522] [0.20869839] [3.30235437]
[2.66000112] [0.22385605] [2.9221897]
[2.93315758] [0.24184418] [2.81055629]
[3.13223459] [0.25410223] [2.40431564]
[3.44138829] [0.27027075] [1.34241064]
[3.73824863] [0.27960847] [1.16749801]
[4.06158644] [0.28836096] [1.02753086]
[4.34199206] [0.29495594] [0.38819736]
[4.63988183] [0.29825502] [0.53297104]
[4.94350774] [0.30221848] [1.33638178]
[5.20219326] [0.30922142] [1.40640322]
[5.47151494] [0.31670267] [1.88415769]
[5.79153422] [0.32821331] [2.21095003]
[6.02716975] [0.33797882] [2.27943506]
[6.29964367] [0.34949363] [2.31511721]
[6.53028993] [0.35930495] [2.41692173]
[6.83524921] [0.37274475

CancelledError: 

In [131]:
await node.set_variables(motors(0, 0))

{'error_code': 2}